In [460]:
# This code has two parts:
# 1. For window kernels for Gaussian covariance used in 'Survey_window_kernels.ipynb'
# 2. For window power spectra for super-sample covariance (SSC) used in 'Covariance_SurveyGeometry.ipynb'

In [ ]:
# This code was written jointly by Otavio Alves (oalves@umich.edu) and Jay Wadekar

# Needs nbodykit installed (https://nbodykit.readthedocs.io/en/latest/getting-started/install.html)
# and files for the survey random catalog

In [1]:
import numpy as np
from numpy import exp, log, log10, cos, sin, pi, cosh, sinh , sqrt, amin, amax, mean, dot, power, conj
import dask.array as da
from matplotlib import pyplot as plot
import matplotlib.pyplot as plt
import itertools as itt
import fitsio
from nbodykit.source.catalog import FITSCatalog, CSVCatalog
from nbodykit.lab import *
from scipy import fft

In [2]:
# Download the BOSS random catalog from https://data.sdss.org/sas/dr12/boss/lss/
# RA, Dec, z, w_fkp, nbar are given by columns 0,1,2,3,4 in the fits files
# Reference: https://nbodykit.readthedocs.io/en/latest/catalogs/reading.html?highlight=fits#FITS-Data

randoms = FITSCatalog('/data/jayw/NYU/Covariance/Data/BOSS/catalogs/random0_DR12v5_CMASSLOWZTOT_North.fits')
randoms = randoms[(randoms['Z'] > 0.5) * (randoms['Z'] < 0.75)]

In [3]:
cosmo = cosmology.Cosmology(h=0.7).match(Omega0_m=0.31)

randoms['OriginalPosition'] = transform.SkyToCartesian(
    randoms['RA'], randoms['DEC'], randoms['Z'], degrees=True, cosmo=cosmo)

r = randoms['OriginalPosition'].T

In [4]:
# Calculating Iij

#I22 is calculated as an example here (note difference in the nbar power as nbar also enters in the sum)
I22 = da.sum(randoms['NZ']**1 * randoms['WEIGHT_FKP']**2)
I22.compute()

437.18336496809707

In [15]:
num_ffts = lambda n: int((n+1)*(n+2)/2) # Number of FFTs of nth order

# Window kernels for Gaussian covariance

In [5]:
Nmesh = 48 # FFT size
BoxSize = 3750. #Box size, should encompass all the galaxies

In [6]:
# Shifting the points such that the survey center is in the center of the box
randoms['Position'] = randoms['OriginalPosition'] + da.array(3*[BoxSize/2])

In [7]:
randoms['W12'] = randoms['WEIGHT_FKP']**2 
randoms['W22'] = (randoms['WEIGHT_FKP']**2) * randoms['NZ']

In [8]:
# Types of fourth-order FFTs that will be generated below

w='W22'
label=[]
for (i,i_label),(j,j_label),(k,k_label),(l,l_label) in itt.combinations_with_replacement(enumerate(['x', 'y', 'z']), r=4):
        label.append(w + i_label + j_label + k_label + l_label)

print(', '.join(label))

W22xxxx, W22xxxy, W22xxxz, W22xxyy, W22xxyz, W22xxzz, W22xyyy, W22xyyz, W22xyzz, W22xzzz, W22yyyy, W22yyyz, W22yyzz, W22yzzz, W22zzzz


In [207]:
# Warning: computing this cell takes a bit of time

export=np.zeros((2*(1+num_ffts(2)+num_ffts(4)),Nmesh,Nmesh,Nmesh),dtype='complex128')

ind=0

for w in ['W22', 'W12']:
    print(f'Computing FFTs of {w}')
    
    print('Computing 0th order FFTs')
    Wij = np.fft.fftn(randoms.to_mesh(Nmesh=Nmesh, BoxSize=BoxSize, value=w, resampler='tsc', interlaced=True, compensated=True).paint())
    Wij *= (da.sum(randoms[w]).compute())/np.real(Wij[0,0,0]) #Fixing normalization, e.g., zero mode should be I22 for 'W22'
    export[ind]=Wij; ind+=1
    
    print('Computing 2nd order FFTs')
    for (i,i_label),(j,j_label) in itt.combinations_with_replacement(enumerate(['x', 'y', 'z']), r=2):
        label = w + i_label + j_label
        randoms[label] = randoms[w] * r[i]*r[j] /(r[0]**2 + r[1]**2 + r[2]**2)
        Wij = np.fft.fftn(randoms.to_mesh(Nmesh=Nmesh, BoxSize=BoxSize, value=label, resampler='tsc', interlaced=True, compensated=True).paint())
        Wij *= (da.sum(randoms[label]).compute())/np.real(Wij[0,0,0])
        export[ind]=Wij; ind+=1

    print('Computing 4th order FFTs')
    for (i,i_label),(j,j_label),(k,k_label),(l,l_label) in itt.combinations_with_replacement(enumerate(['x', 'y', 'z']), r=4):
        label = w + i_label + j_label + k_label + l_label
        randoms[label] = randoms[w] * r[i]*r[j]*r[k]*r[l] /(r[0]**2 + r[1]**2 + r[2]**2)**2
        Wij = np.fft.fftn(randoms.to_mesh(Nmesh=Nmesh, BoxSize=BoxSize, value=label, resampler='tsc', interlaced=True, compensated=True).paint())
        Wij *= (da.sum(randoms[label]).compute())/np.real(Wij[0,0,0])
        export[ind]=Wij; ind+=1
        

Computing FFTs of W22
Computing 0th order FFTs
Computing 2nd order FFTs
Computing 4th order FFTs
Computing FFTs of W12
Computing 0th order FFTs
Computing 2nd order FFTs
Computing 4th order FFTs


In [174]:
export.shape

(44, 48, 48, 48)

In [175]:
np.save('/data/jayw/NYU/Covariance/Data/Delete/FFTWinFun_HighZ_NGC.npy',export)

# Window power spectra for SSC

In [16]:
Nmesh = 500 # use an even number to avoid spurious effects
BoxSize = 7200. #Box size, used much larger than previously in order to sample the low-k modes

# Fundamental k-mode
kfun=2.*np.pi/BoxSize

nBins=int(Nmesh/2) # Number of bins in which power spectrum will be calculated

In [ ]:
# Using similar commands as for the earlier case of Gaussian window kernels
# Warning: computing this cell takes a bit of time

randoms['Position'] = randoms['OriginalPosition'] + da.array(3*[BoxSize/2])

randoms['W22'] = (randoms['WEIGHT_FKP']**2) * randoms['NZ']
randoms['W10'] = randoms['W22']/randoms['W22']

export=np.zeros((2*(1+num_ffts(2)+num_ffts(4)),Nmesh,Nmesh,Nmesh),dtype='complex128')

ind=0

for w in ['W22', 'W10']:
    print(f'Computing FFTs of {w}')
    
    print('Computing 0th order FFTs')
    Wij = np.fft.fftn(randoms.to_mesh(Nmesh=Nmesh, BoxSize=BoxSize, value=w, resampler='tsc', interlaced=True, compensated=True).paint())
    Wij *= (da.sum(randoms[w]).compute())/np.real(Wij[0,0,0]) #Fixing normalization, e.g., zero mode should be I22 for 'W22'
    export[ind]=Wij; ind+=1
    
    print('Computing 2nd order FFTs')
    for (i,i_label),(j,j_label) in itt.combinations_with_replacement(enumerate(['x', 'y', 'z']), r=2):
        label = w + i_label + j_label
        randoms[label] = randoms[w] * r[i]*r[j] /(r[0]**2 + r[1]**2 + r[2]**2)
        Wij = np.fft.fftn(randoms.to_mesh(Nmesh=Nmesh, BoxSize=BoxSize, value=label, resampler='tsc', interlaced=True, compensated=True).paint())
        Wij *= (da.sum(randoms[label]).compute())/np.real(Wij[0,0,0])
        export[ind]=Wij; ind+=1

    print('Computing 4th order FFTs')
    for (i,i_label),(j,j_label),(k,k_label),(l,l_label) in itt.combinations_with_replacement(enumerate(['x', 'y', 'z']), r=4):
        label = w + i_label + j_label + k_label + l_label
        randoms[label] = randoms[w] * r[i]*r[j]*r[k]*r[l] /(r[0]**2 + r[1]**2 + r[2]**2)**2
        Wij = np.fft.fftn(randoms.to_mesh(Nmesh=Nmesh, BoxSize=BoxSize, value=label, resampler='tsc', interlaced=True, compensated=True).paint())
        Wij *= (da.sum(randoms[label]).compute())/np.real(Wij[0,0,0])
        export[ind]=Wij; ind+=1

Computing FFTs of W22
Computing 0th order FFTs


In [ ]:
# For shifting the zero-frequency component to the center of the FFT array
export=np.array([np.fft.fftshift(i) for i in export])

In [ ]:
# Recording the k-modes in different shells
# Bin_kmodes contains [kx,ky,kz,radius] values of all the modes in the bin

[kx,ky,kz] = np.zeros((3,Nmesh,Nmesh,Nmesh));

for i in range(len(kx)):
    kx[i,:,:]+=i-Nmesh/2; ky[:,i,:]+=i-Nmesh/2; kz[:,:,i]+=i-Nmesh/2

rk=np.sqrt(kx**2+ky**2+kz**2)
sort=(rk).astype(int)

rk[nBins,nBins,nBins]=1e10; kx/=rk; ky/=rk; kz/=rk; rk[nBins,nBins,nBins]=0 #rk being zero at the center causes issues so fixed that

In [ ]:
# Reading the FFT files for W22 (referred to as W hereafter for brevity) and W10

[W, Wxx, Wxy, Wxz, Wyy, Wyz, Wzz, Wxxxx, Wxxxy, Wxxxz, Wxxyy, Wxxyz, Wxxzz, Wxyyy, Wxyyz, Wxyzz,\
 Wxzzz, Wyyyy, Wyyyz, Wyyzz, Wyzzz, Wzzzz, W10, W10xx, W10xy, W10xz, W10yy, W10yz, W10zz, W10xxxx,\
 W10xxxy, W10xxxz, W10xxyy, W10xxyz, W10xxzz, W10xyyy, W10xyyz, W10xyzz, W10xzzz, W10yyyy, W10yyyz,\
 W10yyzz, W10yzzz, W10zzzz] = export

W_L0 = W
        
W_L2=1.5*(Wxx*kx**2+Wyy*ky**2+Wzz*kz**2+2.*Wxy*kx*ky+2.*Wyz*ky*kz+2.*Wxz*kz*kx)-0.5*W
        
W_L4=35./8.*(Wxxxx*kx**4 +Wyyyy*ky**4+Wzzzz*kz**4 \
     +4.*Wxxxy*kx**3*ky +4.*Wxxxz*kx**3*kz +4.*Wxyyy*ky**3*kx \
     +4.*Wyyyz*ky**3*kz +4.*Wxzzz*kz**3*kx +4.*Wyzzz*kz**3*ky \
     +6.*Wxxyy*kx**2*ky**2+6.*Wxxzz*kx**2*kz**2+6.*Wyyzz*ky**2*kz**2 \
     +12.*Wxxyz*kx**2*ky*kz+12.*Wxyyz*ky**2*kx*kz +12.*Wxyzz*kz**2*kx*ky) \
     -5./2.*W_L2 -7./8.*W_L0

W10_L0 = W10
        
W10_L2=1.5*(W10xx*kx**2+W10yy*ky**2+W10zz*kz**2+2.*W10xy*kx*ky+2.*W10yz*ky*kz+2.*W10xz*kz*kx)-0.5*W10
        
W10_L4=35./8.*(W10xxxx*kx**4 +W10yyyy*ky**4+W10zzzz*kz**4 \
     +4.*W10xxxy*kx**3*ky +4.*W10xxxz*kx**3*kz +4.*W10xyyy*ky**3*kx \
     +4.*W10yyyz*ky**3*kz +4.*W10xzzz*kz**3*kx +4.*W10yzzz*kz**3*ky \
     +6.*W10xxyy*kx**2*ky**2+6.*W10xxzz*kx**2*kz**2+6.*W10yyzz*ky**2*kz**2 \
     +12.*W10xxyz*kx**2*ky*kz+12.*W10xyyz*ky**2*kx*kz +12.*W10xyzz*kz**2*kx*ky) \
     -5./2.*W10_L2 -7./8.*W10_L0

In [ ]:
def PowerCalc(arr):
    _=np.zeros(nBins,dtype='<c8')
    for i in range(nBins):
        ind=(sort==i)
        _[i]=np.average(arr[ind])
    return(np.real(_))

In [ ]:
# Using this rough calculation, one can see that shot noise is indeed very small and can be ignored
shotNoise = da.sum(randoms['NZ']**3*randoms['WEIGHT_FKP']**4).compute()/(da.sum(randoms['W22']).compute())**2
shotNoise

In [ ]:
P_W=np.zeros((22,nBins))

P_W[0]=PowerCalc(rk)*kfun # Mean |k| in the bin

P_W[1]=PowerCalc(W_L0*conj(W_L0)) # P00
P_W[2]=PowerCalc(W_L0*conj(W_L2))*5 # P02
P_W[3]=PowerCalc(W_L0*conj(W_L4))*9 # P04
P_W[4]=PowerCalc(W_L2*conj(W_L2))*25 # P22
P_W[5]=PowerCalc(W_L2*conj(W_L4))*45 # P24
P_W[6]=PowerCalc(W_L4*conj(W_L4))*81 # P44

P_W[7]=PowerCalc(W10_L0*conj(W10_L0)) # P00
P_W[8]=PowerCalc(W10_L0*conj(W10_L2))*5 # P02
P_W[9]=PowerCalc(W10_L0*conj(W10_L4))*9 # P04
P_W[10]=PowerCalc(W10_L2*conj(W10_L2))*25 # P22
P_W[11]=PowerCalc(W10_L2*conj(W10_L4))*45 # P24
P_W[12]=PowerCalc(W10_L4*conj(W10_L4))*81 # P44

P_W[13]=PowerCalc(W_L0*conj(W10_L0)) # P00
P_W[14]=PowerCalc(W_L0*conj(W10_L2))*5 # P02
P_W[15]=PowerCalc(W_L0*conj(W10_L4))*9 # P04
P_W[16]=PowerCalc(W_L2*conj(W10_L0))*5 # P20
P_W[17]=PowerCalc(W_L2*conj(W10_L2))*25 # P22
P_W[18]=PowerCalc(W_L2*conj(W10_L4))*45 # P24
P_W[19]=PowerCalc(W_L4*conj(W10_L0))*9 # P40
P_W[20]=PowerCalc(W_L4*conj(W10_L2))*45 # P42
P_W[21]=PowerCalc(W_L4*conj(W10_L4))*81 # P44


P_W[1:7]/=(da.sum(randoms['W22']).compute())**2
P_W[7:13]/=(da.sum(randoms['W10']).compute())**2
P_W[13:]/=(da.sum(randoms['W10']).compute()*da.sum(randoms['W22']).compute())

# Removing shot noise
P_W[1] -= shotNoise; P_W[7] -= shotNoise; P_W[13] -= shotNoise

# Minor point: setting k=0 modes by hand to avoid spurious values
P_W[1:7,0]=[1,0,0,1,0,1]; P_W[7:13,0]=[1,0,0,1,0,1]; P_W[13:,0]=[1,0,0,0,1,0,0,0,1]

In [ ]:
np.save('/data/jayw/NYU/Covariance/Data/Delete/WindowPowers_highz.npy',P_W)